### fMRI learn project - Pre-processing steps

#### Author: Boqian Shi

In [3]:
# Result csv reading
import sys
sys.path.append('/home/bqshi/workspace/fmri_topolearn')
sys.path.append('/home/bqshi/workspace/fmri_topolearn/src')
import subjects
print(subjects.results)




List of subjects to be downloaded:
[]

List of subjects to be converted into BIDs format:
['129S4369', '136S4848', '100S4970', '136S0186', '130S4817', '130S2391', '031S2018', '031S4032', '013S2324', '100S4512', '130S4982', '018S4868', '019S4285', '018S4257', '130S5059', '130S4605', '130S4984', '130S4971', '018S4809', '053S4578', '002S4746', '019S4548', '031S4721', '019S4293', '136S4932', '130S4343', '018S4399', '136S4836', '053S4813', '130S4990', '010S5163', '018S4696', '130S4415', '013S4985', '136S4408', '129S4073', '010S4135', '010S4442', '031S4194', '031S4590', '129S4422', '012S4987', '130S5006', '136S4433', '018S4733', '031S4005', '006S4679', '002S4654', '006S4150', '002S4521', '019S4367', '100S5246', '130S4294', '053S5070', '130S4883', '130S4925', '006S4153', '006S4713', '019S4835', '012S4643', '019S4680', '019S4477', '013S2389', '018S2180', '100S4469', '019S5019', '018S2138', '012S4012', '053S2357', '018S4889', '012S4094', '130S4589', '136S4517', '013S4917', '013S4579', '002S47

In [12]:
import os
import pandas as pd

# Path to the subjects.csv file
subjects_file_path = '/home/bqshi/workspace/fmri_topolearn/data/data_index/subjects.csv'

# Read the subject IDs from the CSV file
subjects_df = pd.read_csv(subjects_file_path, header=None)
subjects = subjects_df[0].tolist()

# Base directory where the '00_dicom' folder resides
base_dir = '/home/bqshi/workspace/fmri_topolearn/data/00_dicom'

# Create each subject's folder and sub-folders
for subject in subjects:
    # Format the subject folder name
    subject_folder = f'sub-{subject.replace("_", "")}'
    subject_path = os.path.join(base_dir, subject_folder)
    
    # Create the main subject folder
    os.makedirs(subject_path, exist_ok=True)
    
    # Create sub-folders for RS-fMRI and T1w DICOM files
    os.makedirs(os.path.join(subject_path, 'RS-fMRI DICOM files'), exist_ok=True)
    os.makedirs(os.path.join(subject_path, 'T1w DICOM files'), exist_ok=True)

print("Folders and subfolders created successfully.")
# mprage_variations = ['MPRAGE', 'M-PRAGE', 'MPRAGE_SENSE2', 'MPRAGE_SENSE', 'MPRAGE_SENSE2_SENSE']


Folders and subfolders created successfully.


In [1]:
import os
import shutil
import glob

# Define the original and new source directories
source_dirs = [
    '/mnt/c/Workspace/ADNI data/Extracted/ADNI',  # Adjusted for WSL2 access
    '/mnt/c/Workspace/ADNI data/Extracted/All_t1w+fmri/ADNI'  # New added folder
]

target_dir = '/home/bqshi/workspace/fmri_topolearn/data/00_dicom'

# MPRAGE variations 
mprage_variations = ['MPRAGE', 'M-PRAGE', 'MPRAGE_SENSE2', 'MPRAGE_SENSE', 'MPRAGE_SENSE2_SENSE']


def is_dir_empty(dir):
    """Check if a directory is empty."""
    return not any(os.listdir(dir))

def copy_contents(src, dest):
    """
    Copy all contents of src directory to dest directory if dest is empty.
    """
    if is_dir_empty(dest):
        for item in os.listdir(src):
            s = os.path.join(src, item)
            d = os.path.join(dest, item)
            if os.path.isdir(s):
                shutil.copytree(s, d, dirs_exist_ok=True)
            else:
                shutil.copy2(s, d)
    else:
        print(f"Skipping {dest}, as it is not empty.")

def find_and_copy_files(src, dest, patterns):
    """
    Find directories matching patterns and copy their contents to dest if dest is empty.
    """
    for pattern in patterns:
        for path in glob.glob(os.path.join(src, f"**/{pattern}"), recursive=True):
            if os.path.isdir(path):
                # Copy the contents of the directory if dest is empty
                copy_contents(path, dest)

# Process the subjects and their data
subject_count = sum([len(os.listdir(source_dir)) for source_dir in source_dirs])
processed_count = 0

for source_dir in source_dirs:
    # Iterate through each subject folder in the source directory
    for subject_folder in os.listdir(source_dir):
        subject_path = os.path.join(source_dir, subject_folder)
        if os.path.isdir(subject_path):
            # Format the target subject folder path
            subject_id = subject_folder.replace('_', '')
            target_subject_folder = f'sub-{subject_id}'
            target_path = os.path.join(target_dir, target_subject_folder)
            
            # Target directories for T1w and RS-fMRI
            t1w_dir = os.path.join(target_path, 'T1w DICOM files')
            rsfmri_dir = os.path.join(target_path, 'RS-fMRI DICOM files')
            if not os.path.exists(t1w_dir):
                os.makedirs(t1w_dir, exist_ok=True)
            if not os.path.exists(rsfmri_dir):
                os.makedirs(rsfmri_dir, exist_ok=True)
            
            # Copy the contents of MPRAGE folders if T1w dir is empty
            find_and_copy_files(subject_path, t1w_dir, mprage_variations)
            
            # Copy Resting_State_fMRI files/directories if RS-fMRI dir is empty
            find_and_copy_files(subject_path, rsfmri_dir, ["Resting_State_fMRI"])
        
        processed_count += 1
        print(f"Processed {processed_count}/{subject_count} subjects. Skipped copying if target not empty.")

print("Data transfer complete.")


Skipping /home/bqshi/workspace/fmri_topolearn/data/00_dicom/sub-002S0295/T1w DICOM files, as it is not empty.
Skipping /home/bqshi/workspace/fmri_topolearn/data/00_dicom/sub-002S0295/T1w DICOM files, as it is not empty.
Skipping /home/bqshi/workspace/fmri_topolearn/data/00_dicom/sub-002S0295/RS-fMRI DICOM files, as it is not empty.
Processed 1/370 subjects. Skipped copying if target not empty.
Skipping /home/bqshi/workspace/fmri_topolearn/data/00_dicom/sub-002S0413/T1w DICOM files, as it is not empty.
Skipping /home/bqshi/workspace/fmri_topolearn/data/00_dicom/sub-002S0413/T1w DICOM files, as it is not empty.
Skipping /home/bqshi/workspace/fmri_topolearn/data/00_dicom/sub-002S0413/RS-fMRI DICOM files, as it is not empty.
Processed 2/370 subjects. Skipped copying if target not empty.
Skipping /home/bqshi/workspace/fmri_topolearn/data/00_dicom/sub-002S0685/T1w DICOM files, as it is not empty.
Skipping /home/bqshi/workspace/fmri_topolearn/data/00_dicom/sub-002S0685/T1w DICOM files, as it 